In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\kenneth\AppData\Local\Temp\ipykernel_22128\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import string
from nltk.corpus import stopwords
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer
import nltk

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [4]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\kenneth\Desktop\IronHack AI Engineer\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [5]:
data_train, data_val, y_train, y_val = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)


## Data Preprocessing

In [6]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [36]:
import re

def clean_html(text):

    # Remove HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)

    # Remove URLs
    text = re.sub(r'http\S+', ' ', text)  
    text = re.sub(r'www\S+', ' ', text)   

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [37]:
def clean_text(text):
    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # Remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Remove single characters appearing at the start
    text = re.sub(r'^\s*[a-zA-Z]\s+', ' ', text)

    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # Convert to lowercase
    return text.lower().strip()

In [11]:
data_train_cleaned = data_train.apply(clean_text)
data_val_cleaned = data_val.apply(clean_text)

## Now let's work on removing stopwords
Remove the stopwords.

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

# Apply to cleaned training and validation
data_train_cleaned = data_train_cleaned.apply(remove_stopwords)
data_val_cleaned = data_val_cleaned.apply(remove_stopwords)

print(data_train_cleaned.head())  

29     regards mr nelson smith kindly reply private e...
535           able reach oscar supposed send pdb receive
695    huma abedin checking pat work jack jake rest a...
557                               announced monday today
836    bank africaagence san pedro bp san pedro cote ...
Name: text, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [ ]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Apply lemmatization
data_train_cleaned = data_train_cleaned.apply(lemmatize_text)
data_val_cleaned = data_val_cleaned.apply(lemmatize_text)

print(data_train_cleaned.head())


29     regard mr nelson smith kindly reply private em...
535           able reach oscar supposed send pdb receive
695    huma abedin checking pat work jack jake rest a...
557                               announced monday today
836    bank africaagence san pedro bp san pedro cote ...
Name: text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
def get_top_words(series, n=10):
    all_text = ' '.join(series)  
    words = all_text.split()  
    word_counts = {}  

    for word in words:
        word_counts[word] = word_counts.get(word, 0) + 1  

    sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)  # Sort by frequency
    return sorted_words[:n]  # Return top n words

train_data = pd.DataFrame({'text': data_train_cleaned, 'label': y_train})  # Combine text and labels into DataFrame

# Filter based on integer labels (0 = ham, 1 = spam)
ham_words = get_top_words(train_data[train_data['label'] == 0]['text'])  
spam_words = get_top_words(train_data[train_data['label'] == 1]['text'])  


print("Top 10 words in HAM messages:", ham_words)
print("Top 10 words in SPAM messages:", spam_words)


Top 10 words in HAM messages: [('â', 201), ('state', 142), ('pm', 97), ('would', 94), ('ã', 89), ('president', 89), ('mr', 89), ('time', 81), ('percent', 80), ('obama', 77)]
Top 10 words in SPAM messages: [('money', 847), ('account', 743), ('u', 650), ('bank', 646), ('fund', 626), ('br', 600), ('e', 502), ('transaction', 471), ('business', 424), ('mr', 422)]


## Extra features

In [ ]:
data_train['preprocessed_text'] = data_train_cleaned
data_val['preprocessed_text'] = data_val_cleaned

In [21]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on the 5...
557    I can have it announced here on Monday - can't...
836        BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...
Name: text, dtype: object

## How would work the Bag of Words with Count Vectorizer concept?

In [22]:
count_vectorizer = CountVectorizer(max_features=5000)
X_train_bow = count_vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_bow = count_vectorizer.transform(data_val['preprocessed_text'])

print(f"Bag of Words shape for training set: {X_train_bow.shape}")
print(f"Bag of Words shape for validation set: {X_val_bow.shape}")

Bag of Words shape for training set: (800, 5000)
Bag of Words shape for validation set: (200, 5000)


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [23]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_tfidf = tfidf_vectorizer.transform(data_val['preprocessed_text'])

print(f"TF-IDF shape for training set: {X_train_tfidf.shape}")
print(f"TF-IDF shape for validation set: {X_val_tfidf.shape}")

TF-IDF shape for training set: (800, 5000)
TF-IDF shape for validation set: (200, 5000)


## And the Train a Classifier?

In [27]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def train_and_evaluate(X_train, X_val, y_train, y_val, feature_name):

    # Classifier
    clf = MultinomialNB()
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_val)

    # Evaluate 
    accuracy = accuracy_score(y_val, y_pred)
    conf_matrix = confusion_matrix(y_val, y_pred)
    class_report = classification_report(y_val, y_pred)


    print("="*50)
    print(f"Model Performance using {feature_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nConfusion Matrix:")
    print(conf_matrix)
    print("\nClassification Report:")
    print(class_report)
    print("="*50)

    return clf, accuracy

clf_bow, acc_bow = train_and_evaluate(X_train_bow, X_val_bow, y_train, y_val, "Bag of Words")


clf_tfidf, acc_tfidf = train_and_evaluate(X_train_tfidf, X_val_tfidf, y_train, y_val, "TF-IDF")


Model Performance using Bag of Words:
Accuracy: 0.9800

Confusion Matrix:
[[122   3]
 [  1  74]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       125
           1       0.96      0.99      0.97        75

    accuracy                           0.98       200
   macro avg       0.98      0.98      0.98       200
weighted avg       0.98      0.98      0.98       200

Model Performance using TF-IDF:
Accuracy: 0.9750

Confusion Matrix:
[[120   5]
 [  0  75]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       125
           1       0.94      1.00      0.97        75

    accuracy                           0.97       200
   macro avg       0.97      0.98      0.97       200
weighted avg       0.98      0.97      0.98       200



**Model Performance**

Bag of Words Accuracy: 98.00%
TF-IDF Accuracy: 97.50%

Confusion matrix show that it often predicts correctly fake or not fake.

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [33]:
# Trying three different vectorization methods
def extract_features(method, train_texts, val_texts, max_features=5000):
    if method == "BoW":
        vectorizer = CountVectorizer(max_features=max_features)
    elif method == "TF-IDF":
        vectorizer = TfidfVectorizer(max_features=max_features)
    elif method == "Bi-TFIDF":
        vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=(1,2))  # Bi-grams
    else:
        raise ValueError("Invalid method")

    X_train = vectorizer.fit_transform(train_texts)
    X_val = vectorizer.transform(val_texts)

    return X_train, X_val, vectorizer

# Train and evaluate the classifier (Multinomial)
def train_and_evaluate(X_train, X_val, y_train, y_val, method):
    clf = MultinomialNB()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)

    print(f"Model Performance using {method}: Accuracy = {accuracy:.4f}")
    return accuracy


results = {}

for feature_method in ["BoW", "TF-IDF", "Bi-TFIDF"]:
    X_train, X_val, vectorizer = extract_features(feature_method, data_train['preprocessed_text'], data_val['preprocessed_text'])
    acc = train_and_evaluate(X_train, X_val, y_train, y_val, feature_method)
    results[feature_method] = acc

best_method = max(results, key=results.get)
print(f"\nBest Feature Representation: {best_method} with Accuracy = {results[best_method]:.4f}")

Model Performance using BoW: Accuracy = 0.9800
Model Performance using TF-IDF: Accuracy = 0.9750
Model Performance using Bi-TFIDF: Accuracy = 0.9850

Best Feature Representation: Bi-TFIDF with Accuracy = 0.9850
